<center><h1>Validación cruzada</h1></center>

## 1. Introducción

La validación de entrenamiento/prueba es una técnica simple para probar la precisión de un modelo de aprendizaje automático en datos nuevos en los que el modelo no fue entrenado. Ahora, nos centraremos en técnicas más sólidas.

Para empezar, nos centraremos en la técnica de **validación por exclusión (holdout validation)**, que implica:

- dividir el conjunto de datos completo en 2 particiones:
    - un conjunto de entrenamiento
    - un conjunto de prueba
- entrenar el modelo en el conjunto de entrenamiento,
- usar el modelo entrenado para predecir etiquetas en el conjunto de prueba,
- calcular una métrica de error para comprender la efectividad del modelo,
- cambiar los conjuntos de entrenamiento y prueba y repita,
- promediar los errores.

En la validación por exclusión, generalmente usamos una división 50/50 en lugar de la división 75/25 de la validación de entrenamiento/prueba. De esta forma, eliminamos el número de observaciones como fuente potencial de variación en el rendimiento de nuestro modelo.

<img src="figs/holdout_validation.png" width="600" height="400" />


Comencemos por dividir el conjunto de datos en 2 mitades casi equivalentes.

Cuando divida el conjunto de datos, no olvide configurar una copia usando `.copy()` para asegurarse de no obtener resultados inesperados más adelante. Si ejecuta el código localmente en Jupyter Notebook o Jupyter Lab sin `.copy()`, notará lo que se conoce como Advertencia de configuración con copia. Esto no evitará que su código se ejecute correctamente, pero le permite saber que cualquier operación que esté haciendo está tratando de establecerse en una copia de un segmento de un dataframe. Para asegurarse de que no vea esta advertencia, asegúrese de incluir `.copy()` cada vez que realice operaciones en un dataframe.

### Ejercicio

- Utilice la función `numpy.random.permutation()` para aleatorizar el orden de las filas en `dc_listings`.
- Seleccione las primeras 1862 filas y asígnelas a `split_one`.
- Seleccione las 1861 filas restantes y asígnelas a `split_two`.

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
dc_listings = pd.read_csv("dc_airbnb.csv")

In [3]:
dc_listings['price'] = dc_listings['price'].str.replace('[\$,]', '', regex=True).astype('float')

print(len(dc_listings) / 2, '\n')

1861.5 



In [4]:
dc_listings = dc_listings.iloc[np.random.default_rng(2021).permutation(len(dc_listings))]

In [5]:
split_one = dc_listings.iloc[:1862].copy()
split_two = dc_listings.iloc[1862:].copy()

## 2. Holdout Validation
Ahora que hemos dividido nuestro conjunto de datos en 2 dataframes, hagamos lo siguiente:

- entrenar un modelo de k-vecinos más cercanos en la primera mitad,
- probar este modelo en la segunda mitad,
- entrenar un modelo de k-vecinos más cercanos en la segunda mitad,
- probar este modelo en la primera mitad.

### Ejercicio

- Entrene un modelo de k-vecinos más cercanos utilizando el algoritmo predeterminado (`auto`) y el número predeterminado de vecinos (`5`) que:
     - Utiliza la columna `accommodates` de `train_one` para entrenar y
     - Lo prueba en `test_one`.
- Asigne el valor RMSE resultante a `iteration_one_rmse`.
- Entrene un modelo de k-vecinos más cercanos utilizando el algoritmo predeterminado (`auto`) y el número predeterminado de vecinos (`5`) que:
     - Utiliza la columna `accommodates` de `test_two` para entrenar y
     - Lo prueba en `test_two`.
- Asigne el valor RMSE resultante a `iteration_two_rmse`.
- Use `numpy.mean()` para calcular el promedio de los 2 valores RMSE y asigne a `avg_rmse`.

In [6]:
train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

In [7]:
knn1 = KNeighborsRegressor()
knn1.fit(train_one[['accommodates']], train_one[['price']])
predictions1 = knn1.predict(test_one[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(predictions1, test_one['price']))

In [8]:
knn2 = KNeighborsRegressor()
knn2.fit(train_two[['accommodates']], train_two['price'])
predictions2 = knn2.predict(test_two[['accommodates']])
iteration_two_rmse = np.sqrt(mean_squared_error(predictions2, test_two['price']))

In [9]:
print(iteration_one_rmse)
print(iteration_two_rmse)

128.70692056945418
125.33395047116689


In [10]:
avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])
print(avg_rmse)

127.02043552031054


## 3. Validación cruzada K-Fold

Si promediamos los dos valores RMSE del último paso, obtenemos un valor RMSE de aproximadamente 127.02. La validación por exclusión (holdout validation) es en realidad un ejemplo específico de una clase más grande de técnicas de validación llamada validación cruzada k-fold. 

Si bien la validación de exclusión es mejor que la validación de entrenamiento/prueba porque el modelo no está sesgado repetidamente hacia un subconjunto específico de datos, ambos modelos entrenados solo usan la mitad de los datos disponibles. La validación cruzada K-fold, por otro lado, aprovecha una mayor proporción de los datos durante el entrenamiento mientras sigue rotando a través de diferentes subconjuntos de datos para evitar los problemas de validación de entrenamiento/prueba.

Aquí está el algoritmo de la validación cruzada de k-fold:

- dividir el conjunto de datos completo en `k` particiones de igual longitud.
    - seleccionando las `k-1` particiones  como el conjunto de entrenamiento y
    - seleccionando la partición restante como el conjunto de prueba
- entrenar el modelo en el conjunto de entrenamiento.
- usar el modelo entrenado para predecir etiquetas en el conjunto de prueba.
- calcular la métrica de error del conjunto prueba.
- repetir todos los pasos anteriores `k-1` veces, hasta que cada partición se haya utilizado como conjunto de prueba para una iteración.
- calcular la media de los valores de error `k`.

La validación de exclusión es esencialmente una versión de la validación cruzada k-fold cuando `k` es igual a `2`. En general, los pliegues (número de subconjuntos) de `5` o `10` se utilizan para la validación cruzada k-fold. Aquí hay un diagrama que describe cada iteración de la validación cruzada de 5 subconjuntos:

<img src="figs/kfold_cross_validation.png" width="800" height="600" />

A medida que aumenta el número de pliegues, el número de observaciones en cada pliegue (subconjunto) disminuye y la varianza de los errores entre cada pliegue aumenta. 

Comencemos dividiendo manualmente el conjunto de datos en 5 pliegues. En lugar de dividir en 5 dataframes, agreguemos una columna que especifique a qué subconjunto pertenece la fila. De esta manera, podemos seleccionar fácilmente nuestro conjunto de entrenamiento y nuestro conjunto de prueba.

### Ejercicio

- Agregue una nueva columna a `dc_listings` llamada `fold` que contiene el número de subconjunto al que pertenece cada fila:
- El pliegue `1` debe tener filas desde el índice `0` hasta `745`, sin incluir `745`.
- El pliegue `2` debe tener filas desde el índice `745` hasta `1490`, sin incluir `1490`.
- El pliegue `3` debe tener filas desde el índice `1490` hasta `2234`, sin incluir `2234`.
- El pliegue `4` debe tener filas desde el índice `2234` hasta `2978`, sin incluir `2978`.
- El pliegue `5` debe tener filas desde el índice `2978` hasta `3723`, sin incluir `3723`.
- Asegúrate de que el tipo de `fold` sea un tipo flotante.
- Cuente los valores únicos para la columna `fold` para confirmar que cada pliegue tiene aproximadamente la misma cantidad de elementos.
- Muestra el número de valores faltantes (missing values) en la columna `fold` para confirmar que no nos falta ninguna fila.

In [11]:
dc_listings.loc[dc_listings.index[:745], 'fold'] = 1.0

In [12]:
dc_listings['fold']

604     1.0
2493    1.0
904     1.0
1935    1.0
544     1.0
       ... 
2127    NaN
3388    NaN
1769    NaN
1333    NaN
3412    NaN
Name: fold, Length: 3723, dtype: float64

In [13]:
dc_listings.loc[dc_listings.index[745:1490], 'fold'] = 2
dc_listings.loc[dc_listings.index[1490:2234], 'fold'] = 3
dc_listings.loc[dc_listings.index[2234:2978], 'fold'] = 4
dc_listings.loc[dc_listings.index[2978:], 'fold'] = 5

In [14]:
dc_listings['fold'].value_counts().sort_index()

1.0    745
2.0    745
3.0    744
4.0    744
5.0    745
Name: fold, dtype: int64

In [15]:
# c_listings['fold'] = 1.0
# dc_listings.fold.iloc[745:1490] = 2
# dc_listings.fold.iloc[1490:2234] = 3
# dc_listings.fold.iloc[2234:2978] = 4
# dc_listings.fold.iloc[2978:] = 5

In [16]:
print(dc_listings['fold'].isnull().sum())

0


## 4. Primera iteración

Comencemos por realizar la primera iteración de la validación cruzada de k-fold en un modelo univariado.

### Ejercicio

- Entrene un modelo de k-vecinos más cercanos usando la columna `accommodates` como la única característica de los pliegues `2` a `5` como conjunto de entrenamiento.
- Use el modelo para hacer predicciones en el conjunto de prueba (columna `accommodates` del subconjunto `1`) y asigne las etiquetas pronosticadas a `labels`.
- Calcule el valor RMSE comparando la columna `price` con las etiquetas pronosticadas.
- Asigne el valor RMSE a `iteration_one_rmse`.

In [17]:
print(dc_listings[dc_listings['fold'] != 1]['fold'].unique())

[2. 3. 4. 5.]


In [18]:
training_df = dc_listings[dc_listings['fold'] != 1].copy()
test_df = dc_listings[dc_listings['fold'] == 1].copy()

In [19]:
knn = KNeighborsRegressor()
knn.fit(training_df[['accommodates']], training_df['price'])
labels = knn.predict(test_df[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(labels, test_df['price']))

In [20]:
print(iteration_one_rmse)

111.45677104635922


## 5. Función para entrenar modelos

Desde la primera iteración, logramos un valor RMSE de aproximadamente **111**. Calculemos los valores RMSE para las iteraciones restantes. Para facilitar el proceso de iteración, coloquemos el código que escribimos anteriormente en una función.


### Ejercicio

- Escriba una función llamada `train_and_validate` que tome un dataframe como primer parámetro (`df`) y una lista de valores de subconjuntos (`1` a `5` en nuestro caso) como segundo parámetro (`folds`). Esta función debería:
    - Entrenar modelos `n` (donde `n` es el número de subconjuntos) y realizar una validación cruzada k-fold (usando `n` subconjuntos). Utilice el valor `k` predeterminado para la clase `KNeighborsRegressor`.
    - Devuelva una lista de valores RMSE, donde el primer elemento es el RMSE cuando el subconjunto `1` fue el conjunto de prueba, el segundo elemento es el RMSE para cuando el subconjunto `2` fue el conjunto de prueba, y así sucesivamente.
- Use la función `train_and_validate` para devolver la lista de valores RMSE para el dataframe `dc_listings` y asigne a `rmses`.
- Calcular la media de estos valores y asignar a `avg_rmse`.
- Mostrar tanto `rmses` como `avg_rmse`.

In [21]:
def train_and_validate(df, folds):
    rmses = []
    
    for n in folds:
        training_df = df[df['fold'] != n].copy()
        test_df = df[df['fold'] == n].copy()

        knn = KNeighborsRegressor() 
        knn.fit(training_df[['accommodates']], training_df['price'])

        labels = knn.predict(test_df[['accommodates']])
        rmses.append(np.sqrt(mean_squared_error(labels, test_df['price'])))
    return rmses

In [22]:
rmses = train_and_validate(dc_listings, folds=[1, 2, 3, 4, 5])
avg_rmse = np.mean(rmses)

In [23]:
print(rmses)
print(avg_rmse)

[111.45677104635922, 123.78106565531378, 178.30716142994456, 153.87714325088623, 155.0998542182535]
144.50439912015148


## 6. Realización de la validación cruzada K-Fold con Scikit-Learn

Si bien el valor promedio de RMSE fue de aproximadamente **135**, los valores de RMSE oscilaron entre **102** y **159+**. Esta gran cantidad de variabilidad entre los valores RMSE significa que estamos usando un modelo deficiente o un criterio de evaluación deficiente (¡o un poco de ambos!). Al implementar su propia función de validación cruzada k-fold, es de esperar que hayas adquirido una buena comprensión del funcionamiento interno de la técnica. 

La función que escribimos, sin embargo, tiene muchas limitaciones. Si ahora queremos cambiar la cantidad de pliegues(subconjuntos) que deseamos usar, debemos hacer que la función sea más general para que también pueda manejar la ordenación aleatoria de las filas en el dataframe y la división en subconjuntos.

En el aprendizaje automático, estamos interesados en construir un buen modelo y comprender con precisión qué tan bien funcionará. Para construir un mejor modelo de k-vecinos más cercanos, podemos cambiar las características(variables) que usa o ajustar la cantidad de vecinos (un hiperparámetro). Para comprender con precisión el rendimiento de un modelo, podemos realizar una validación cruzada k-fold y seleccionar el número adecuado de pliegues. Hemos aprendido cómo scikit-learn nos facilita experimentar rápidamente con estas diferentes configuranciones cuando se trata de construir un mejor modelo. Ahora profundicemos en cómo podemos usar scikit-learn para manejar también la validación cruzada.

Primero, instanciamos la [clase KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) desde `sklearn.model_selection`:

```python
from sklearn.model_selection import KFold
kf = KFold(n_splits, shuffle=False, random_state=None)
```

donde:

- `n_splits` es el número de pliegues que desea utilizar,
- `shuffle` se usa para cambiar aleatorizar(revolver) las observaciones en el conjunto de datos,
- `random_state` se usa para especificar el valor inicial aleatorio si `shuffle` se establece en `True`.

Notará aquí que ningún parámetro depende en absoluto del conjunto de datos. Esto se debe a que la clase KFold devuelve un iterador que usamos junto con la función [cross_val_score()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), también de `sklearn.model_selection`. Juntas, estas 2 funciones nos permiten entrenar y probar de forma compacta utilizando la validación cruzada k-fold:

Estos son los parámetros relevantes para la función `cross_val_score`:

```python
from sklearn.model_selection import cross_val_score
cross_val_score(estimador, X, Y, scoring=None, cv=None)
```

donde:

- `estimator` es un modelo sklearn que implementa el método `fit` (por ejemplo, instancia de KNeighborsRegressor),
- `X` es la lista o matriz 2D que contiene las funciones en las que desea entrenar,
- `y` es una lista que contiene los valores que desea predecir (columna de destino),
- `scoring` es una cadena(texto) que describe los criterios de puntuación (lista de valores aceptados <a href="https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values" target="_blank">aquí</a>).
- `cv` describe el número de pliegues(subconjuntos). Estos son algunos ejemplos de valores aceptados:
    - una instancia de la clase `KFold`,
    - un número entero que representa el número de pliegues.

Dependiendo de los criterios de puntuación que especifique, se devuelve un único valor total para cada pliegue. Aquí está el flujo de trabajo general para realizar la validación cruzada k-fold usando las clases que acabamos de describir:

- crear una instancia de la clase de modelo de scikit-learn que desea ajustar,
- instanciar la clase `KFold` y usar los parámetros para especificar los atributos de validación cruzada k-fold que desea,
- utilice la función `cross_val_score()` para devolver la métrica de puntuación que le interesa.


### Ejercicio

- Crear una nueva instancia de la clase `KFold` con las siguientes propiedades:
    - `5` subconjuntos,
    - reproducción aleatoria establecida en 'Verdadero',
    - semilla aleatoria establecida en `1` (para que podamos verificar usando la misma semilla),
    - asignar a la variable `kf`.
- Crear una nueva instancia de la clase `KNeighborsRegressor` y asignarla a `knn`.
- Use la función `cross_val_score()` para realizar una validación cruzada k-fold:
    - usar la instancia de KNeighborsRegressor `knn`,
    - usar la columna `accommodates` para entrenamiento,
    - utilizar la columna `price` como la columna de destino(target),
    - usar la cadena `neg_mean_squared_error` como el valor del parámetro `scoring`,
    - usar `kf` como el valor del parámetro `cv`
    - devolver una arreglo de valores MSE (un valor para cada pliegue o subconjunto).
- Asigne la lista resultante de valores MSE a `mses`. Luego, tome el valor absoluto seguido de la raíz cuadrada de cada valor de MSE. Luego, calcule el promedio de los valores RMSE resultantes y asigne a `avg_rmse`.

In [24]:
from sklearn.model_selection import cross_val_score, KFold

In [25]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()

In [26]:
mses = cross_val_score(estimator=knn, 
                       X=dc_listings[['accommodates']], 
                       y=dc_listings['price'], 
                       scoring='neg_mean_squared_error', 
                       cv=kf)

In [27]:
avg_rmse = np.sqrt(np.abs(mses)).mean()
print(avg_rmse)

132.53435892114908


## 7. Explorando diferentes valores de K

Elegir el valor `k` correcto al realizar una validación cruzada k-fold es más un arte que una ciencia. Como discutimos anteriormente, un valor `k` de `2` es realmente solo una validación de exclusioń (holdout validation). Por otro lado, establecer `k` igual a `n` (el número de observaciones en el conjunto de datos) se conoce como **leave-one-out cross validation**, o LOOCV para abreviar. Es común tomar `10` como el valor `k` estándar.

A continuacioń se muestran los resultados de variar `k` de `3` a `23`. Para cada valor `k`, calculamos y mostramos el valor RMSE promedio en todos los subconjuntos y la desviación estándar de los valores RMSE. A través de los diferentes valores de `k`, parece que el valor RMSE promedio es de alrededor de `129`. Notará que la desviación estándar del RMSE aumenta de aproximadamente `8` a más de `40` a medida que aumentamos el número de pliegues.

In [28]:
num_folds = [3, 5, 7, 9, 10, 11, 13, 15, 17, 19, 21, 23]

for fold in num_folds:
    kf = KFold(fold, shuffle=True, random_state=1)
    model = KNeighborsRegressor()
    mses = cross_val_score(model, dc_listings[["accommodates"]], dc_listings["price"], scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(np.absolute(mses))
    avg_rmse = np.mean(rmses)
    std_rmse = np.std(rmses)
    print(f'{fold} folds: avg RMSE: {avg_rmse}, std RMSE: {std_rmse}')

3 folds: avg RMSE: 126.25643788844208, std RMSE: 2.0462220503302757
5 folds: avg RMSE: 132.53435892114908, std RMSE: 10.583222443315243
7 folds: avg RMSE: 138.84369852094946, std RMSE: 8.54990777432938
9 folds: avg RMSE: 125.79037301186888, std RMSE: 21.189505646505364
10 folds: avg RMSE: 130.20650881456987, std RMSE: 23.963967671753206
11 folds: avg RMSE: 126.88756397456419, std RMSE: 25.758906948602437
13 folds: avg RMSE: 126.70440036154892, std RMSE: 29.05210451728983
15 folds: avg RMSE: 130.2583160888913, std RMSE: 30.686863668342742
17 folds: avg RMSE: 128.6152830146413, std RMSE: 35.26730539015145
19 folds: avg RMSE: 126.7894453710179, std RMSE: 34.63276931942381
21 folds: avg RMSE: 126.65264051559065, std RMSE: 37.987028538782695
23 folds: avg RMSE: 125.11583067337774, std RMSE: 40.59600544050042


## 8. Equilibrio entre sesgo y varianza

Hemos estado trabajando bajo el supuesto de que un RMSE más bajo siempre significa que un modelo es más preciso. Sin embargo, un modelo tiene dos fuentes de error, **sesgo** y **varianza**.

El sesgo o bias describe el error que da como resultado malas suposiciones sobre el algoritmo de aprendizaje. Por ejemplo, suponiendo que solo una característica (variable), como el peso de un automóvil, se relaciona con la eficiencia de combustible de un automóvil, lo llevará a ajustar un modelo de regresión univariante simple que dará como resultado un sesgo alto. La tasa de error será alta ya que la eficiencia de combustible de un automóvil se ve afectada por muchos otros factores además de su peso.

La varianza describe el error que ocurre debido a la variabilidad de los valores predichos de un modelo. Si recibimos un conjunto de datos con 1000 características en cada automóvil y usamos cada una de las características para entrenar un modelo de regresión multivariante increíblemente complicado, tendremos un sesgo bajo pero una varianza alta. En un mundo ideal, queremos un sesgo bajo y una varianza baja pero, en realidad, siempre hay tradeoff.

La *desviación estándar* de los valores RMSE puede ser un indicador de la **varianza** de un modelo, mientras que el *RMSE promedio* es un indicador del **sesgo** de un modelo. El sesgo y la varianza son las 2 fuentes de error observables en un modelo que podemos controlar indirectamente.


<img src="figs/bias_variance.png" width="600" height="400" />

Si bien los k-vecinos más cercanos pueden hacer predicciones, no es un modelo matemático. Un modelo matemático suele ser una ecuación que puede existir sin los datos originales, lo que no ocurre con los k vecinos más cercanos. 

## 9. Conclusión

Exploramos técnicas de validación cruzada más sólidas, como la validación por exlusión y la validación cruzada k-fold.
